In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

In [2]:
iris = load_iris()
x = iris.data
y = iris.target

In [3]:
# 붓꽃의 꽃받침과 꽃잎의 [길이, 너비]에 대한 값 -> 붓꽃의 종류 setosa / versicolor / virginica
print(x.shape, y.shape)

(150, 4) (150,)


In [4]:
print(x[0:10], y[0:10])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]] [0 0 0 0 0 0 0 0 0 0]


In [ ]:
np.max(x, axis=0)

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x) 

In [ ]:
print(x[0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = x_train.reshape(-1, 30, 4)
y_train = y_train.reshape(-1, 30)

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [ ]:
class FCN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FCN, self).__init__()
        self.fc = nn.Linear(input_dim, 4)
        self.fc2 = nn.Linear(4, num_classes)

    def forward(self, x):
        x = torch.sigmoid(self.fc(x))
        return torch.sigmoid(self.fc2(x))

In [ ]:
model = FCN(input_dim=x_train.shape[2], num_classes=3)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 10000
for e in range(epochs):
    for b in range(len(x_train)):
        outputs = model(x_train[b])
        loss = criterion(outputs, y_train[b])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (e+1) % 1000 == 0:
        print(f'Epoch {e + 1} / {epochs}, Loss : {loss.item()}')

In [ ]:
# 테스트
with torch.no_grad():

    model.eval()

    correct = 0
    total = 0
    outputs = model(x_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
    print('Accuracy of the model on test data: {} %'.format(100 * correct / total))
